In [1]:
from tools import pretraitement, buildmodel
import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

2021-12-08 20:18:44.937318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-08 20:18:44.937408: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Chargement du jeu de données
(x_train_origin, y_train_origin), (x_test_origin, y_test_origin) = tf.keras.datasets.cifar10.load_data()

# pretraitement des données
(x_train, y_train) = pretraitement.normalize_dataset(x_train_origin, y_train_origin)

(x_test, y_test) = pretraitement.normalize_dataset(x_test_origin, y_test_origin)

del x_train_origin
del x_test_origin
del y_train_origin
del y_test_origin

assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 10)
assert y_test.shape == (10000, 10)

# Hyper paramètres

Dans ce notebook, nous passons à la recherche des paramètres d'entrainement qui donnent le meilleurs résultas. Dans ce qui suit, nous prenons une seule structure de modèle ainsi qu'une certaine augmentation des données. Ces derniers points ont été traités dans de précedant notebook.

In [4]:
parametres = { 'data': {
    'x_train_shape': x_train.shape,
    'y_train_shape': y_train.shape,
    'x_test_shape': x_test.shape,
    'y_test_shape': y_test.shape,
    'augmented_data': True
}}

datagen, parametres['data']['augmentation_param'] = buildmodel.create_datagen()

## Variation des paramètre

- learning_rate prend ces valeurs dans : `[0.005, 0.001, 0.0001]`
- loss prend ces valeurs dans : `['mean_squared_error', 'categorical_crossentropy']`
- optimizer prend ces valeurs dans : `['adam', '...']`
- dropout: `[ ]`


In [9]:
# Compiling model
def compil_model(lr=0.001, loss='categorical_crossentropy',
                opt='adam',epochs=40, batch_size=64,dropout=0.5):
    
    model = buildmodel.create_model(dropout=dropout)
    
    optimizers={'adam':Adam(learning_rate=lr)}
    optimizer = 'adam'
    metrics=['accuracy']
    
    model.compile(optimizer=optimizers[optimizer],
                  loss=loss,
                  metrics=metrics)
    
    return model

, {
        'learning_rate': lr,
        'loss': loss,
        'optimizer': optimizer,
        'metrics': metrics,
        'epochs': epochs,
        'batch_size': batch_size
    }

In [10]:
def gridSearch(x_train, y_train,x_test, y_test, verbose=False):
    param = {
        'loss': ['categorical_crossentropy','mean_squared_error'],
        'lr': [0.005, 0.001, 0.0001],
        'dropout': [0.1, 0.3, 0.5],
        'opt': ['adam']
    }
    # create model
    model = KerasClassifier(build_fn=compil_model, verbose=2)
    
    gd_sr = GridSearchCV(estimator=model,  
                         param_grid=param,
                         scoring='accuracy',
                         cv=2,
                         n_jobs=-1,
                        verbose=2)
    
    gd_sr.fit(np.append(x_train, x_test, axis=0), 
                        np.append(y_train,y_test,axis=0))
    if verbose:
        print("Best score: "+str(gd_sr.best_score_))
    
    predictions = (name, gd_sr.best_score_, gd_sr.best_estimator_,gd_sr.best_params_)
        
    del gd_sr
    
    return predictions

In [ ]:
gridSearch(x_train, y_train, x_test, y_test)

2021-12-08 20:22:07.835092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-08 20:22:07.835072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-08 20:22:07.835184: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-08 20:22:07.835199: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
